In [1]:
import alphaspace2 as al
import mdtraj
import numpy as np
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import fcluster, linkage
from alphaspace2.functions import _binCluster, _group
from alphaspace2.Cluster import _DPocket

In [2]:
from communities import genCommunityPocket,CoreCluster

In [3]:
import features

In [4]:
## applying community detection for full CDK2 surface ##

In [4]:
cdk2_prot = mdtraj.load('CDK2_Communities/cdk2_prot.pdb')
al.annotateVinaAtomTypes(pdbqt="CDK2_Communities/cdk2_prot.pdbqt", receptor=cdk2_prot)

Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file found, trimming last {-1} entries
Redundant Atom types in pdbqt file

In [5]:
ss_cdk2 = al.Snapshot()
ss_cdk2.run(cdk2_prot)

In [6]:
temp_space_dict = {}
temp_coords_dict = {}
temp_contact_dict = {}
temp_score_dict = {}
temp_centroids_dict = {}
for px,pocket in enumerate(ss_cdk2.pockets):
    temp_coords_dict[px] = [list(b.xyz) for b in pocket.betas]
    temp_score_dict[px] = np.array([min(b.scores) for b in  pocket.betas])
    temp_space_dict[px] = np.array([b.space for b in  pocket.betas])
    temp_contact_dict[px] = list(pocket.lining_atoms_idx)
    temp_centroids_dict[px] = pocket.centroid


In [8]:
prot_coords = cdk2_prot.xyz[0]*10   ### change nm to Angstrom

In [9]:
### generate communities
surface_communities = genCommunityPocket(prot_coords, temp_centroids_dict, temp_space_dict, \
                                            temp_contact_dict, temp_score_dict, \
                                            corecut = 100, auxcut = 30, tight_option = True, tight_cutoff_core = 12.5, tight_cutoff_aux = 6.5)

In [13]:
### calculate more features of beta clusters community ###
surface_communities_props = {}
for cx,community in surface_communities.items():
    temp_coords_array = []
    for pock in community['core_pockets'] + community['aux_pockets']:
        temp_coords_array.extend(temp_coords_dict[pock])
    temp_coords_array = np.array(temp_coords_array)
    volume = features._get_grid_volume(temp_coords_array)
    occluded_asa = features._get_pharmacophore_fingerprint(cdk2_prot,temp_coords_array)
    surface_communities_props[cx] = {}
    surface_communities_props[cx]['space'] = community['space']
    surface_communities_props[cx]['score'] = community['score']
    surface_communities_props[cx]['volume'] = volume
    surface_communities_props[cx]['occluded_asa'] = occluded_asa['Total_OASA']    
    

In [18]:
for cx,props in surface_communities_props.items():
    print(cx,'Space : '+str(props['space']), ', Score : '+str(props['score']), \
          ', Volume : ' +str(props['volume']),', Occluded ASA : '+str(props['occluded_asa']))

0 Space : 1472.0 , Score : -23.69 , Volume : 690.25 , Occluded ASA : 619.090723156929
1 Space : 1199.0 , Score : -18.26 , Volume : 574.125 , Occluded ASA : 675.7055418863893
2 Space : 889.0 , Score : -14.36 , Volume : 474.5 , Occluded ASA : 516.9363194331527
3 Space : 885.0 , Score : -18.98 , Volume : 564.625 , Occluded ASA : 538.0038401335478
4 Space : 659.0 , Score : -10.99 , Volume : 413.875 , Occluded ASA : 504.916808180511
5 Space : 647.0 , Score : -14.08 , Volume : 392.625 , Occluded ASA : 371.4489370509982
6 Space : 457.0 , Score : -10.53 , Volume : 317.25 , Occluded ASA : 349.74282832443714
7 Space : 406.0 , Score : -8.39 , Volume : 199.5 , Occluded ASA : 224.78633822500706
8 Space : 351.0 , Score : -5.82 , Volume : 167.75 , Occluded ASA : 188.64039415121078
9 Space : 335.0 , Score : -6.33 , Volume : 187.0 , Occluded ASA : 233.435670286417
10 Space : 285.0 , Score : -5.62 , Volume : 140.75 , Occluded ASA : 203.43154790997505
11 Space : 283.0 , Score : -7.18 , Volume : 261.75 , 